In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to compute edge image using Sobel or Laplacian
def compute_edge_image(img, method='gradient'):
    if method == 'gradient':
        # Use Sobel to compute gradient magnitude
        grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        edge_img = np.sqrt(grad_x**2 + grad_y**2)
    elif method == 'laplacian':
        # Use Laplacian
        edge_img = cv2.Laplacian(img, cv2.CV_64F)
        edge_img = np.abs(edge_img)
    else:
        raise ValueError("Unknown method. Use 'gradient' or 'laplacian'.")
    return edge_img

# Function to compute and plot the histogram
def compute_histogram(img, title='Histogram'):
    # Compute and plot the histogram of the image
    hist, bins = np.histogram(img.flatten(), bins=256, range=[0, 256])
    plt.figure()
    plt.plot(hist, color='black')
    plt.xlim([0, 256])
    plt.title(title)
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.show()

# Function to perform variable thresholding
def variable_thresholding(img, initial_threshold, max_iterations=10):
    current_threshold = initial_threshold
    
    for i in range(max_iterations):
        print(f"Iteration {i+1}, Current Threshold: {current_threshold}")
        
        # Step 1: Threshold the image using the current threshold
        _, binary_img = cv2.threshold(img, current_threshold, 255, cv2.THRESH_BINARY)
        
        # Plot the histogram of the current thresholded image
        compute_histogram(binary_img, title=f'Histogram after Iteration {i+1}')

        # Calculate mean of background and foreground
        background = img[binary_img == 0]
        foreground = img[binary_img == 255]
        
        if len(background) == 0 or len(foreground) == 0:
            print("No more pixels to segment, stopping iterations.")
            break

        mean_background = np.mean(background) if len(background) > 0 else 0
        mean_foreground = np.mean(foreground) if len(foreground) > 0 else 0
        
        # Step 2: Update the threshold as the average of the means
        new_threshold = (mean_background + mean_foreground) / 2
        
        # Stop if the threshold value stops changing significantly
        if abs(new_threshold - current_threshold) < 1:
            print(f"Converged after {i+1} iterations, Final Threshold: {new_threshold}")
            break
        
        current_threshold = new_threshold
    
    # Return final thresholded image and threshold
    return binary_img, current_threshold

# Load and preprocess the image
img = cv2.imread('sc1', cv2.IMREAD_GRAYSCALE)

# Step 1: Compute edge image (optional)
edge_img = compute_edge_image(img, method='gradient')

# Step 2: Perform variable thresholding
initial_threshold = 50  # You can adjust this initial value
final_binary_img, final_threshold = variable_thresholding(edge_img, initial_threshold)

# Step 3: Display the original image and the final thresholded image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1), plt.imshow(img, cmap='gray'), plt.title('Original Image')
plt.subplot(1, 2, 2), plt.imshow(final_binary_img, cmap='gray'), plt.title(f'Final Thresholded Image (T={final_threshold})')
plt.show()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:419: error: (-215:Assertion failed) !_src.empty() in function 'cv::Sobel'
